# Scalability Benchmark

In [1]:
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from utils.plotting_defaults import (
    set_style,
    parse_logs_wide,
)

set_style()
Path("plots/scalability_benchmark").mkdir(parents=True, exist_ok=True)

In [2]:
try:
    shutil.rmtree(Path.home() / ".cache" / "matplotlib" / "tex.cache")
except FileNotFoundError:
    pass

## Data Loading

In [3]:
execution_list = []
for logfile in Path("../logs/scalability_benchmark/execution/").iterdir():
    config = logfile.stem.split("-")
    data = parse_logs_wide(logfile)

    data["dataset"] = config[0]
    data["query_set"] = config[1]
    data["index_type"] = config[2]
    data["execution"] = config[3]
    data["scaling_factor"] = int(config[4][2:]) / 100

    execution_list.append(data)

execution = pd.DataFrame(
    execution_list,
    columns=[
        "dataset",
        "query_set",
        "index_type",
        "execution",
        "scaling_factor",
        "query_collection_time",
        "query_times",
        "avg_result_size",
    ],
)
execution["query_times_sum"] = execution["query_times"].apply(np.sum)

In [4]:
indexing_list = []
for logfile in Path("../logs/scalability_benchmark/indexing/").iterdir():
    config = logfile.stem.split("-")
    data = parse_logs_wide(logfile)

    data["dataset"] = config[0]
    data["phase"] = config[1]
    data["scaling_factor"] = int(config[2][2:]) / 100

    indexing_list.append(data)

indexing = pd.DataFrame(
    indexing_list,
    columns=[
        "dataset",
        "phase",
        "scaling_factor",
        "total_time",
        "index_size",
    ],
)

## Analysis

### Execution

In [5]:
execution.groupby(["dataset", "query_set", "index_type", "execution", "scaling_factor"]).agg(
    {
        "query_collection_time": ["mean", "std"],
        "query_times_sum": ["mean", "std"],
        "avg_result_size": ["mean"],
    }
)

query_collection_time  \
                                                                                  mean   
dataset   query_set  index_type execution         scaling_factor                         
gittables collection conversion single            0.25                       63.467779   
                                                  0.50                      145.634594   
                                                  1.00                      352.906263   
                                                  2.00                      752.376484   
                                single_suppressed 0.25                        0.292523   
                                                  0.50                        0.306344   
                                                  1.00                        0.324416   
                                                  2.00                        0.340009   
                     rebinning  single            0.25                       54.789540   
                                                  0.50                      126.356023   
                                                  1.00                      329.238724   
                                                  2.00                      697.486478   
                                single_suppressed 0.25                        0.316530   
                                                  0.50                        0.291223   
                                                  1.00                        0.320823   
                                                  2.00                        0.366340   
          query      conversion single            0.25                        0.082565   
                                                  0.50                        0.176898   
                                                  1.00                        0.379609   
                                                  2.00                        0.748370   
                                single_suppressed 0.25                        0.000855   
                                                  0.50                        0.000875   
                                                  1.00                        0.001007   
                                                  2.00                        0.001013   
                     rebinning  single            0.25                        0.082617   
                                                  0.50                        0.172827   
                                                  1.00                        0.364881   
                                                  2.00                        0.806380   
                                single_suppressed 0.25                        0.000873   
                                                  0.50                        0.000869   
                                                  1.00                        0.001004   
                                                  2.00                        0.000998   

                                                                            \
                                                                       std   
dataset   query_set  index_type execution         scaling_factor             
gittables collection conversion single            0.25            0.346856   
                                                  0.50            1.143060   
                                                  1.00            1.828558   
                                                  2.00            3.422099   
                                single_suppressed 0.25            0.003555   
                                                  0.50            0.015042   
                                                  1.00            0.006346   
                                                  2.00            0.003060   
                     rebinning  single            0.25  

In [6]:
height = 1.5
for i, index_type in enumerate(["rebinning", "conversion"]):
    fig, ax = plt.subplots(1, 1, figsize=(1.72, height))
    data = (
        execution[
            (execution["index_type"] == index_type) & (execution["query_set"] == "collection")
        ]
        .groupby(["execution", "scaling_factor"])
        .agg({"query_collection_time": ["mean", "std"]})
        .values
    )

    x = [0.25, 0.5, 1, 2]
    ax.plot(
        x,
        data[: len(x), 0],
        color=sns.color_palette()[0],
        label="w/ results",
    )
    ax.fill_between(
        x,
        data[: len(x), 0] - data[: len(x), 1],
        data[: len(x), 0] + data[: len(x), 1],
        alpha=0.1,
        edgecolor="white",
        color=sns.color_palette()[0],
    )
    ax.plot(
        x,
        data[len(x) :, 0],
        color=sns.color_palette()[1],
        label="w/o results",
    )
    ax.fill_between(
        x,
        data[len(x) :, 0] - data[len(x) :, 1],
        data[len(x) :, 0] + data[len(x) :, 1],
        alpha=0.1,
        edgecolor="white",
        color=sns.color_palette()[1],
    )

    ax.set_xticks(x, labels=[".25", ".5", "1", "2"])
    ax.set_ylabel("Time (s)")
    ax.set_yscale("log")
    ax.set_ylim(top=ax.get_ylim()[1] * 3)
    ax.legend(loc="upper left")
    sns.despine()
    bbox = fig.get_tightbbox()
    plt.tight_layout(pad=1.02)
    plt.savefig(
        f"plots/scalability_benchmark/{index_type}.pdf", bbox_inches="tight", pad_inches=0.01
    )
    plt.close()

handles, labels = ax.get_legend_handles_labels()  # type: ignore

### Index Construction

In [7]:
indexing.groupby(["dataset", "phase", "scaling_factor"]).mean()

total_time    index_size
dataset   phase      scaling_factor                           
gittables clustering 0.25            1587.253769           NaN
                     0.50            3368.694606           NaN
                     1.00            6933.406293           NaN
                     2.00            4893.088123           NaN
          conversion 0.25             286.604510   8556.341976
                     0.50             605.006456  17552.750676
                     1.00            1049.702077  29508.362532
                     2.00            2062.994869  58435.706196
          rebinning  0.25             111.063035   4285.651122
                     0.50             225.043177   8791.455630
                     1.00             380.397724  14784.286980
                     2.00             714.502973  29278.064526

**Finding:** The index construction time and size scales linearly with the dataset size, which is consistent with our conceptual discussion.